## Задача 3
UA-менеджер обратился к Вам с просьбой проверить корректность атрибуции пользователей на канале googleadwords_int. У него есть подозрения, что некоторым пользователям не присваивается корректная атрибуция. Проведите исследование данной проблемы и опишите результаты.

## План:

- Загрузить users.csv и attribution.csv в pandas.DataFrame.
- Посчитать:
  - количество пользователей в users
  - количество пользователей, имеющих запись в attribution
  - количество пользователей без атрибуции
- Отфильтровать строки, где media_source == "googleadwords_int", и посчитать их количество и долю от всех пользователей с атрибуцией.
- Найти пользователей, которые есть в users, но отсутствуют в attribution.
- Вывести итоговую таблицу: user_id | есть_в_users | есть_в_attribution | media_source
- На основе таблицы посчитать:
  - сколько пользователей без атрибуции (есть_в_attribution == False)
  - сколько пользователей с googleadwords_int и их долю среди всех с атрибуцией
- Сделать выводы о корректности атрибуции.

In [1]:
import pandas as pd

# Загружаем данные
users = pd.read_csv('users.csv')
attribution = pd.read_csv('attribution.csv')

In [2]:
# Количество пользователей всего
total_users = users['user_id'].nunique()

In [3]:
# Количество пользователей с атрибуцией
users_with_attr = attribution['user_id'].nunique()

In [4]:
# Количество без атрибуции
users_without_attr = total_users - users_with_attr

In [5]:
print("Всего пользователей:", total_users)
print("С атрибуцией:", users_with_attr)
print("Без атрибуции:", users_without_attr)

Всего пользователей: 29767
С атрибуцией: 20711
Без атрибуции: 9056


In [6]:
# Фильтруем пользователей с media_source == 'googleadwords_int'
google_users = attribution[attribution['media_source'] == 'googleadwords_int']

# Количество таких пользователей
google_count = google_users['user_id'].nunique()

# Доля от всех пользователей с атрибуцией
share = round(google_count / users_with_attr * 100, 2)

print("Пользователей с googleadwords_int:", google_count)
print("Доля от всех с атрибуцией:", f"{share}%")


Пользователей с googleadwords_int: 1457
Доля от всех с атрибуцией: 7.03%


In [7]:
# Соединяем users и attribution по user_id
merged = users.merge(
    attribution[['user_id', 'media_source']],
    on='user_id',
    how='left'
)

# Добавляем флаги
merged['есть_в_users'] = True
merged['есть_в_attribution'] = merged['media_source'].notna()

# Оставляем только нужные столбцы
result = merged[['user_id', 'есть_в_users', 'есть_в_attribution', 'media_source']]

# Посмотреть первые строки
result.head()


,user_id,есть_в_users,есть_в_attribution,media_source
0,5feceb66-ffc8-6f38-d952-786c6d696c79,True,True,tiktokglobal_int
1,6b86b273-ff34-fce1-9d6b-804eff5a3f57,True,True,tiktokglobal_int
2,d4735e3a-265e-16ee-e03f-59718b9b5d03,True,True,tiktokglobal_int
3,4e074085-62be-db8b-60ce-05c1decfe3ad,True,True,tiktokglobal_int
4,4b227777-d4dd-1fc6-1c6f-884f48641d02,True,True,tiktokglobal_int


In [8]:
# Всего пользователей
total = result['user_id'].nunique()

# С атрибуцией
with_attr = result[result['есть_в_attribution']].shape[0]

# Без атрибуции
without_attr = result[~result['есть_в_attribution']].shape[0]
without_attr_pct = round(without_attr / total * 100, 2)

# С googleadwords_int
google = result[result['media_source'] == 'googleadwords_int'].shape[0]
google_pct = round(google / with_attr * 100, 2)

print(f"Всего пользователей: {total}")
print(f"С атрибуцией: {with_attr}")
print(f"Без атрибуции: {without_attr} ({without_attr_pct}%)")
print(f"С googleadwords_int: {google} ({google_pct}% от всех с атрибуцией)")

Всего пользователей: 29767
С атрибуцией: 20711
Без атрибуции: 9056 (30.42%)
С googleadwords_int: 1457 (7.03% от всех с атрибуцией)


## Выводы

- Доля пользователей без атрибуции — 30%, что довольно много и может указывать на проблему с обработкой событий установки.
- Доля пользователей с googleadwords_int — 7%, что выглядит довольно скромно.
- Возможно, часть пользователей, которые пришли с googleadwords_int, не получили атрибуцию и попали в группу «без атрибуции».

Есть признаки проблемы с атрибуцией: высокая доля пользователей без атрибуции может искажать статистику по `googleadwords_int` и занижать его эффективность.